In [ ]:
!pip install oracledb sentence-transformers 

In [3]:
import oracledb

# 1. Connect to Oracle
conn = oracledb.connect(
    user="system",                  #
    password="apelix",              # 
    dsn="localhost:1521/freepdb1"   # default PDB for Oracle 23ai Free
)
print("Connected to Oracle 23ai!")

cursor = conn.cursor()

# 2. Create a simple table
try:
    cursor.execute("DROP TABLE employees PURGE")
except Exception:
    pass

cursor.execute("""
    CREATE TABLE employees (
        id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
        name VARCHAR2(50),
        salary NUMBER
    )
""")
print("Table created.")

# 3. Insert data
cursor.execute("INSERT INTO employees (name, salary) VALUES (:1, :2)", ("Alice", 50000))
cursor.execute("INSERT INTO employees (name, salary) VALUES (:1, :2)", ("Bob", 60000))
conn.commit()
print("Data inserted.")

# 4. Query data
cursor.execute("SELECT id, name, salary FROM employees")
for row in cursor.fetchall():
    print(row)

# 5. Update data
cursor.execute("UPDATE employees SET salary = salary + 5000 WHERE name = :1", ("Alice",))
conn.commit()

# 6. Query again
cursor.execute("SELECT id, name, salary FROM employees")
print("\nAfter update:")
for row in cursor.fetchall():
    print(row)

# 7. Delete data
cursor.execute("DELETE FROM employees WHERE name = :1", ("Bob",))
conn.commit()

cursor.execute("SELECT id, name, salary FROM employees")
print("\nAfter delete:")
for row in cursor.fetchall():
    print(row)

# 8. Clean up
cursor.close()
conn.close()
print("\nConnection closed.")


Connected to Oracle 23ai!
Table created.
Data inserted.
(1, 'Alice', 50000)
(2, 'Bob', 60000)

After update:
(1, 'Alice', 55000)
(2, 'Bob', 60000)

After delete:
(1, 'Alice', 55000)

Connection closed.


In [6]:
import oracledb
import json
from sentence_transformers import SentenceTransformer

# Connect to Oracle
conn = oracledb.connect(user="system", password="apelix", dsn="localhost:1521/freepdb1")
cursor = conn.cursor()
print("Connected to Oracle 23ai!")

# Check tablespaces with automatic segment space management
cursor.execute("""
    SELECT tablespace_name 
    FROM dba_tablespaces 
    WHERE segment_space_management='AUTO'
""")
auto_tablespaces = [row[0] for row in cursor.fetchall()]

if not auto_tablespaces:
    raise Exception("No tablespace with AUTO segment space management found.")

tablespace_to_use = auto_tablespaces[0]
print("Using tablespace:", tablespace_to_use)

# Embedding model config
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
EMBED_DIM = 384
embedder = SentenceTransformer(EMBED_MODEL)

# Drop table if exists
try:
    cursor.execute("DROP TABLE documents PURGE")
except Exception:
    pass

# Create VECTOR table in AUTO tablespace
cursor.execute(f"""
    CREATE TABLE documents (
        id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
        content CLOB,
        embedding VECTOR({EMBED_DIM})
    )
    TABLESPACE {tablespace_to_use}
""")

# Create VECTOR index
#
# cursor.execute(f"""
#    CREATE INDEX documents_embedding_idx
#    ON documents (embedding)
# INDEXTYPE IS VECTOR_INDEX
# """)
#
conn.commit()
print("VECTOR table created successfully in tablespace:", tablespace_to_use)

# Optional: Insert demo document
def insert_doc(text):
    vec = embedder.encode([text])[0].tolist()
    vec_str = json.dumps(vec)
    cursor.execute("""
        INSERT INTO documents (content, embedding)
        VALUES (:1, TO_VECTOR(:2))
    """, (text, vec_str))
    conn.commit()

insert_doc("Oracle 23ai VECTOR similarity search demo.")
print("Demo document inserted.")

cursor.close()
conn.close()


Connected to Oracle 23ai!
Using tablespace: SYSAUX
VECTOR table created successfully in tablespace: SYSAUX
Demo document inserted.


In [8]:
import oracledb
import json
from sentence_transformers import SentenceTransformer

# ---------------------------
# Configuration
# ---------------------------
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
EMBED_DIM = 384

# Connect to Oracle 23ai (thin mode)
conn = oracledb.connect(
    user="system",
    password="apelix",
    dsn="localhost:1521/freepdb1"
)
cursor = conn.cursor()
print("Connected to Oracle 23ai!")

# Find a tablespace with AUTO segment management
cursor.execute("""
    SELECT tablespace_name 
    FROM dba_tablespaces 
    WHERE segment_space_management='AUTO'
""")
auto_tablespaces = [row[0] for row in cursor.fetchall()]
if not auto_tablespaces:
    raise Exception("No AUTO tablespace found")
tablespace_to_use = auto_tablespaces[0]
print("Using tablespace:", tablespace_to_use)

# ---------------------------
# Create VECTOR table
# ---------------------------
try:
    cursor.execute("DROP TABLE documents PURGE")
except Exception:
    pass

cursor.execute(f"""
    CREATE TABLE documents (
        id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
        content CLOB,
        embedding VECTOR({EMBED_DIM})
    )
    TABLESPACE {tablespace_to_use}
""")
conn.commit()
print("VECTOR table created successfully (no index in thin mode).")

# ---------------------------
# Initialize embedding model
# ---------------------------
embedder = SentenceTransformer(EMBED_MODEL)

# ---------------------------
# Helper functions
# ---------------------------
def insert_doc(text: str):
    vec = embedder.encode([text])[0].tolist()
    vec_str = json.dumps(vec)
    cursor.execute("""
        INSERT INTO documents (content, embedding)
        VALUES (:1, TO_VECTOR(:2))
    """, (text, vec_str))
    conn.commit()
    print("Inserted:", text[:60])

def search_similar(query: str, top_k: int = 2):
    qvec = embedder.encode([query])[0].tolist()
    qvec_str = json.dumps(qvec)
    cursor.execute("""
        SELECT content
        FROM documents
        ORDER BY embedding <-> TO_VECTOR(:1)
        FETCH FIRST :2 ROWS ONLY
    """, [qvec_str, top_k])
    return [row[0] for row in cursor.fetchall()]

# ---------------------------
# Insert demo documents
# ---------------------------
insert_doc("Oracle 23ai supports native VECTOR similarity search for AI applications.")
insert_doc("Python can connect to Oracle 23ai using the oracledb driver.")
insert_doc("Retrieval Augmented Generation (RAG) uses vector databases for context retrieval.")
insert_doc("AI embeddings allow semantic search across documents.")

# ---------------------------
# Run a similarity search
# ---------------------------
query = "How can I search vectors in Oracle 23ai?"
results = search_similar(query, top_k=3)

print("\nQuery:", query)
print("Top results:")
for r in results:
    print("-", r)

# ---------------------------
# Clean up
# ---------------------------
cursor.close()
conn.close()
print("\nConnection closed.")


Connected to Oracle 23ai!
Using tablespace: SYSAUX
VECTOR table created successfully (no index in thin mode).
Inserted: Oracle 23ai supports native VECTOR similarity search for AI 
Inserted: Python can connect to Oracle 23ai using the oracledb driver.
Inserted: Retrieval Augmented Generation (RAG) uses vector databases f
Inserted: AI embeddings allow semantic search across documents.

Query: How can I search vectors in Oracle 23ai?
Top results:
- Oracle 23ai supports native VECTOR similarity search for AI applications.
- Python can connect to Oracle 23ai using the oracledb driver.
- AI embeddings allow semantic search across documents.

Connection closed.
